# 3. Feature Engineering

본격적으로 Feature Engineering을 시작하고자 한다.
먼저 numpy, pandas, matplotlib, seaborn과 같은 라이브러리를 가져오는 기본 세팅을 시작한다. 그 다음 해당 자료를 불러온다. 타이타닉 튜토리얼 1에서 했던 방법 그대로 하면 되니 복습하듯이 따라쳐보자.

In [1]:
import numpy as np
import pandas as pd
from pandas import Series
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5) 
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1 # 자신을 포함해야하니 1을 더합니다
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1 # 자신을 포함해야하니 1을 더합니다

df_test.loc[df_test.Fare.isnull(), 'Fare'] = df_test['Fare'].mean()

df_train['Fare'] = df_train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

### 3.1 Fill Null

#### 3.1.1 Fill Null in Age using title

Age column에는 null data가 177개나 있었다. 이를 채울 수 있는 여러 아이디어가 있겠지만, 여기서 우리는 title + statistics를 사용할 것이다.

영어에서는 Miss, Mr, Mrs 같은 title이 존재한다. 각 탑승객의 이름에는 꼭 이런 title이 들어가게 되는데, 이를 사용해 보자.

pandas series에는 data를 string으로 바꿔주는 str method와 정규표현식을 적용시켜 주는 extract method가 있다. 이를 사용하여 title을 쉽게 추출할 수 있다. title을 initial column에 저장하도록 하겠다.

In [2]:
 df_train['Name']

# 이름을 불러와 보니 이름마다 Mr, Mrs, Miss와 같은 title이 존재함을 알 수 있다.
# 이를 토대로 대략적인 나이를 예측하여 이를 수치로 나타낼 것이다.

In [3]:
df_train['Name'].str.extract('([A-Za-z]+)\.')

In [4]:
df_train['Initial'] = df_train.Name.str.extract('([A-Za-z]+)\.')
df_test['Initial'] = df_test.Name.str.extract('([A-Za-z]+)\.')

pandas의 crosstab을 이용해 우리가 추출한 initial과 sex 간의 count를 살펴보자.

In [5]:
pd.crosstab(df_train['Initial'], df_train['Sex']).T.style.background_gradient(cmap="summer_r")

위 table을 참고하여 남자, 여자가 쓰는 initial을 구분해보고자 한다. replace 메소드를 사용하면, 특정 데이터 값을 1대 1 대응시켜 원하는 값으로 치환해 줄 수 있다.

In [6]:
df_train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

df_test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

In [7]:
df_train.groupby('Initial').mean()

여성과 관계있는 Miss, Mr, Mrs가 생존율이 높은 것을 확인할 수 있다.

In [8]:
df_train.groupby('Initial')['Survived'].mean().plot.bar()

이제 본격적으로 Null을 채울 건데, 우선 null data를 채우는 방법은 정말 많이 존재한다.
statistics를 활용하는 방법도 있고, null data가 없는 데이터를 기반으로 새로운 머신러닝 알고리즘을 만들어 예측해서 채워넣는 방법도 있다.
여기서는 statistics를 활용하는 방법을 사용할 것이다.

여기서 statistics는 train data의 것을 의미한다. 우리는 언제나 test를 unseen으로 둔 상태로 둬야 하며, 
train에서 얻은 statistics를 기반으로 test의 null data를 채워줘야 한다.

In [9]:
df_train.groupby('Initial').mean()

Age의 평균을 이용해 Null data를 채우도록 해보자.
pandas dataframe을 다룰 땐 boolean array를 이용해 indexing하는 방법을 쓰면 된다.
아래 코드 첫 줄을 해석해 보자면, isnull() 이면서 Initial이 Mr인 조건을 만족하는 row(탑승객)의 'Age'의 값을 33으로 치환한다는 뜻이다.
loc + boolean + column을 사용해 값을 치환하는 방법은 자주 쓰이므로 꼭 익숙해져야 한다.

In [10]:
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Mr'),'Age'] = 33
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Mrs'),'Age'] = 36
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Master'),'Age'] = 5
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Miss'),'Age'] = 22
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Other'),'Age'] = 46

df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Mr'),'Age'] = 33
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Mrs'),'Age'] = 36
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Master'),'Age'] = 5
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Miss'),'Age'] = 22
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Other'),'Age'] = 46

In [11]:
# Null 값들이 모두 제거됐는지 확인하는 구문.
df_train.Age.isnull().any()

In [12]:
df_test.Age.isnull().any()

여기선 간단하게 Null을 채웠지만, 좀 더 다양한 방법을 쓴 예시들이 다른 커널에 존재한다. 확인차 df_train.Age와 df_test.Age의 Null Data가 모두 사라졌는지를 나타내는 구문을 통해 확인해 보니 False라고 뜨면서 이제 Null Data가 모두 사라졌다는 것도 알게 되었다.

#### 3.1.2 Fill Null in Embarked

In [13]:
print('Embarked has ', sum(df_train['Embarked'].isnull()), 'Null Values')

Embarked 는 Null value 가 2개이고, S에서 가장 많은 탑승객이 있었으므로, 간단하게 Null을 S로 채우고자 한다. 가장 많이 탑승했던 S로 채우는 게 가장 속 편할 것이다.
dataframe 의 fillna method 를 이용하면 쉽게 채울 수 있다. 여기서 inplace=True 로 하면 df_train 에 fillna 를 실제로 적용한다.

In [14]:
df_train['Embarked'].fillna('S', inplace=True)

In [15]:
df_train.Embarked.isnull().any()       # 이제 Embarked 컬럼에도 Null Data가 모두 채워졌다.

### 3.2 Change Age(continuous to categorical)

나이는 현재 continuous feature이다. 이대로 써도 모델을 세울 수 있지 않느냐 하겠지만 Age를 몇 개의 group으로 나누어 카테고리화 시키는 것이 모델을 세울 때 조금 더 쉽게 할 수 있다. Continuous를 categorical로 바꾸면 자칫 정보 손실이 생길 수도 있지만, 본 튜토리얼에서는 다양한 방법을 소개하는 것이 목적이므로 일단 카테고리를 바꾸는 작업을 진행해 보고자 한다.  

방법은 여러가지가 있는데, loc 를 사용하여 직접해줄 수 있고, 아니면 apply 를 사용해 함수를 넣어줄 수도 있다.
첫번째로 loc를 사용한 방법이다. loc는 DataFrame index를 활용함으로써 행과 열을 지정하여 data를 획득하는 구문이다. loc는 자주쓰게 되므로 그 사용법을 숙지하시면 좋다. 나이는 10살 간격으로 나눈다.



In [16]:
df_train['Age_cat'] = 0
df_train.loc[df_train['Age'] < 10, 'Age_cat'] = 0
df_train.loc[(10 <= df_train['Age']) & (df_train['Age'] < 20), 'Age_cat'] = 1
df_train.loc[(20 <= df_train['Age']) & (df_train['Age'] < 30), 'Age_cat'] = 2
df_train.loc[(30 <= df_train['Age']) & (df_train['Age'] < 40), 'Age_cat'] = 3
df_train.loc[(40 <= df_train['Age']) & (df_train['Age'] < 50), 'Age_cat'] = 4
df_train.loc[(50 <= df_train['Age']) & (df_train['Age'] < 60), 'Age_cat'] = 5
df_train.loc[(60 <= df_train['Age']) & (df_train['Age'] < 70), 'Age_cat'] = 6
df_train.loc[70 <= df_train['Age'], 'Age_cat'] = 7

df_test['Age_cat'] = 0
df_test.loc[df_test['Age'] < 10, 'Age_cat'] = 0
df_test.loc[(10 <= df_test['Age']) & (df_test['Age'] < 20), 'Age_cat'] = 1
df_test.loc[(20 <= df_test['Age']) & (df_test['Age'] < 30), 'Age_cat'] = 2
df_test.loc[(30 <= df_test['Age']) & (df_test['Age'] < 40), 'Age_cat'] = 3
df_test.loc[(40 <= df_test['Age']) & (df_test['Age'] < 50), 'Age_cat'] = 4
df_test.loc[(50 <= df_test['Age']) & (df_test['Age'] < 60), 'Age_cat'] = 5
df_test.loc[(60 <= df_test['Age']) & (df_test['Age'] < 70), 'Age_cat'] = 6
df_test.loc[70 <= df_test['Age'], 'Age_cat'] = 7

두번째로 간단한 함수를 만들어 apply 메소드에 넣어주는 방법이다.  
if와 elif를 구간에 맞게끔 반복 작성하는 것이 포인트이다.  
위의 loc를 썼을 때보다 훨씬 구문이 간단하다.  

In [17]:
def category_age(x):
    if x < 10:
        return 0
    elif x < 20:
        return 1
    elif x < 30:
        return 2
    elif x < 40:
        return 3
    elif x < 50:
        return 4
    elif x < 60:
        return 5
    elif x < 70:
        return 6
    else:
        return 7    
    
df_train['Age_cat_2'] = df_train['Age'].apply(category_age)

두가지 방법이 잘 적용됬다면, 둘다 같은 결과가 나와야 한다.  
이를 확인하기 위해 Series 간 boolean 비교 후 all() 메소드를 확인해 보자.  
all() 메소드는 모든 값이 True면 True, 하나라도 False가 있으면 False를 준다.

In [18]:
print('1번 방법, 2번 방법 둘다 같은 결과를 내면 True 줘야함 -> ', (df_train['Age_cat'] == df_train['Age_cat_2']).all())

보시다시피 True이다. 둘 중 편한 걸 선택하시면 된다.
이제 중복되는 Age_cat 컬럼과 원래 컬럼 Age를 제거한다. 제거를 할 때는 drop 메서드를 활용한다.

In [19]:
df_train.drop(['Age', 'Age_cat_2'], axis=1, inplace=True)
df_test.drop(['Age'], axis=1, inplace=True)

### 3.3. Change Initial, Embarked and Sex(string to numerical)

현재 Initial 은 Mr, Mrs, Miss, Master, Other 총 5개로 이루어져 있다. 이런 문자 형태의 카테고리로 표현되어져 있는 데이터를 모델에 인풋으로 넣어줄 때 우리가 해야할 것은 먼저 컴퓨터가 인식할 수 있도록 수치화 시켜야 한다. map method 를 가지고 간단히 할 수 있다. 사전 순서대로 정리하여 mapping한다.

In [20]:
df_train['Initial'] = df_train['Initial'].map({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs': 3, 'Other': 4})
df_test['Initial'] = df_test['Initial'].map({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs': 3, 'Other': 4})

Embarked 역시 C, Q, S로 이루어져 있으니, map을 이용해 바꾸자.
그러기 앞서서, 특정 column에 어떤 값들이 있는 지 확인해보는 방법을 잠깐 살펴보자면, 간단히 unique() 메소드를 쓰거나, value_counts()를 써서 count까지 보는 방법이 있긴 하다.

In [21]:
# unique 메서드는 지정한 열에서 고유한 값을 추출해내는 기능을 한다.
# 즉 Embarked 열의 고유한 값인 S, C, Q를 추출해 내는 기능을 수행하고 이를 결과로써 확인 가능하다.
df_train['Embarked'].unique()

In [22]:
#  value_count 메서드를 이용해 S, C, Q에서 각각 몇 명이 탑승했는지 직접 카운팅이 가능하다.
df_train['Embarked'].value_counts()

In [23]:
df_train['Embarked'] = df_train['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
df_test['Embarked'] = df_test['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

map 기능을 사용하여 훈련 세트와 테스트 세트의 C는 0으로, Q는 1로, S는 2로 바꾸는 과정이 이루어졌다.  
이 상태에서 Null이 사라졌는지도 확인해보자. Embarked Column만 가져온 것은 하나의 pandas의 Series 객체이므로, isnull() 메서드를 사용해 Series의 값들이 null인지 아닌지에 대한 boolean 값을 얻을 수 있다. 그리고 이것에 any()를 사용해, True가 단 하나라도 있을 시, 즉 Null이 한 개라도 있을 시 True로 반환하며 Null Data가 전혀 없다면 False로 반환한다. 우리는 이미 Embarked 열에 있던 2개의 Null data를 가장 많았던 S로 바꿨기 때문에 null이 없어야 하고 아래 구문을 실행할 시 False로 나와야 할 것이다.

In [24]:
df_train['Embarked'].isnull().any()

In [25]:
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1})
df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1})

위 구문은 Sex도 Male, Female로 이루어져 있는 관계로, map을 이용해 0과 1로 바꾸는 작업에 대한 구문이었다.  
이제 각 feature 간의 상관관계를 한번 보려고 한다. 두 변수간의 상관관계를 구하게 되면 -1부터 1 사이의 값을 얻을 수 있다. -1로 가까이 갈수록 음의 상관관계를 갖게 되며, 1로 갈수록 양의 상관관계를 갖게 된다. 그리고 0으로 가까이 갈수록 상관관계가 전혀 없다는 뜻이다.  
구하는 수식을 아래와 같다.

$r_xy = \frac{Cov(x,y)}{S_xS_y} = \frac{\frac{1}{n-1}\displaystyle\sum_{i=1}^{n}{(x_i - x\bar)(y_i - y\bar)}}{S_xS_y}$  

우리는 여러 feature를 가지고 있으니 이를 하나의 matrix 형태로 보면 편할 텐데, 이를 heatmap plot의 형태로 그려보고자 한다. 이는 dataframe의 corr() 메소드와 seaborn을 가지고 편리하게 할 수 있다.

In [26]:
heatmap_data = df_train[['Survived', 'Pclass', 'Sex', 'Fare', 'Embarked', 'FamilySize', 'Initial', 'Age_cat']] 

colormap = plt.cm.RdBu
plt.figure(figsize=(14, 12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths=0.1, vmax=1.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 16})

del heatmap_data

우리가 EDA에서 살펴봤듯, Sex와 Pclass가 Survived에 상곤관계가 어느 정도 있는 것 같다고 짐작해 볼수 있다. 그리고 생각보다 fare와 Embarked도 상관관계가 있음을 알 수 있다.  
(Sex와 Survived : -0.54, Pclass와 Survived : -0.34, fare와 Embarked : -0.2)  
하지만 우리는 여기서 서로 강한 상관관계를 가지는 feature는 없다는 것 또한 알게 되었고, 이는 곧 우리가 모델을 학습시킬 때 불필요한(redundant, superfluous) feature가 없다는 것을 의미한다. 1 또는 -1의 상관관계를 가진 feature A,B가 있다면, 우리가 얻을 수 있는 정보는 사실 하나일 것이다.  
이제 실제로 모델을 학습시키기 앞서서 data preprocessing(데이터 전처리)을 진행해 보고자 한다.

### 3.4. One-hot encoding on Initial and Embarked

수치화시킨 카테고리 데이터를 그대로 넣어도 되지만, 모델의 성능을 높이기 위해 one-hot encoding을 해줄 수 있다. One-hot encoding은 위 카테고리를 아래와 같이 (0, 1) 로 이루어진 5차원의 벡터로 나타내는 것을 말하며, 수치화는 간단히 Master == 0, Miss == 1, Mr == 2, Mrs == 3, Other == 4 로 매핑해주는 것을 말한다.  
이 작업을 직접 코딩할 수도 있겠지만, pandas의 get_dummies를 사용해 쉽게 해결할 수 있다. 총 5개의 카테고리이니, one-hot encoding을 하고 나면 새로운 5개의 column이 생겨난다. 우선, Initial을 prefix로 두어 구분이 쉽게 만들어준다.

In [27]:
df_train = pd.get_dummies(df_train, columns=['Initial'], prefix='Initial')
df_test = pd.get_dummies(df_test, columns=['Initial'], prefix='Initial')

In [28]:
df_train.head()

위 내용을 실행한 후 표의 맨 우측을 보면 우리가 만들려고 했던 one-hot encoded columns가 생성되어 있는 것을 확인할 수 있다.
Embarked에도 적용해 보고, Initial 때와 마찬가지로 one-hot encoding을 사용해 표현해 보자.

In [29]:
df_train = pd.get_dummies(df_train, columns=['Embarked'], prefix='Embarked')
df_test = pd.get_dummies(df_test, columns=['Embarked'], prefix='Embarked')

아주 쉽게 one-hot encoding을 적용할 수 있었다.  
sklearn(사이킷런)으로 Labelencoder + OneHotencoder 이용해도 one-hot encoding이 가능하다.  
가끔 category가 100개가 넘어가는 경우가 있는데, 이때 one-hot encoding을 사용하면 column이 100개가 생겨, 학습 시 매우 버거울 경우가 있다. 이런 경우는 다른 방법을 사용하기도 한다.

### 3.5. Drop columns

필요한 Column만 남기도 다 지우는 작업을 한다. drop 메서드를 가져와서 이를 수행할 수 있다.

PassengerId : 카테고리화 될 수 없기 때문에 필요없다.  
Name : 카테고리 자료형으로 변환할 수 없으므로 필요없다.  
SibSp와 Parch : FamilySize로 카테고리화를 이미 했기 때문에 필요없다.
Ticket : 이름과 마찬가지로 카테고리화 될 수 없는 무작위 문자열이므로 필요없다.   
Cabin : NaN 값이 너무 많고, 많은 승객에 따라 cabin 값이 많다. 그렇기 때문에 필요없다.  

In [30]:
df_train.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)
df_test.drop(['PassengerId', 'Name',  'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [31]:
df_train.head()

In [32]:
df_test.head()

train과 test의 상위 5개의 데이터와 컬럼을 확인해 보면 Survived feature(target class)를 빼면 train, test 둘다 같은 columns 를 가진 걸 알 수 있다.

# 4. Building machine learning model and prediction using the trained model

데이터 준비가 모두 이루어졌으니, sklearn을 사용해 본격적으로 머신러닝 모델을 만들어 보자.

In [34]:
# 머신러닝 모델을 만들기 위한 패키지를 모두 가져오는 작업
from sklearn.ensemble import RandomForestClassifier     # 유명한 랜덤 포레스트. 
from sklearn import metrics                             # 모델의 평가를 위해서 씁니다
from sklearn.model_selection import train_test_split    # train과 test를 분할하는데 쓰이는 함수.

Sklearn은 머신러닝의 처음부터 끝까지가 다 있다고 할 수 있다. 특성 엔지니어링과 데이터 전처리, 지도 학습 및 비지도 학습 알고리즘, 모델 평가, 파이프라인 등 머신러닝에 관련된 모든 작업들이 손쉬운 인터페이스로 구현되어 있다. 데이터 분석 + 머신러닝을 하고싶다면, 이 라이브러리는 반드시 숙지해야 한다.
파이썬 라이브러리를 활용한 머신러닝(Introduction to machine larning with Python)책을 사서 공부를 하면 크게 도움이 될 거라고 한다.  

지금 타이타닉 문제는 target class로 생존 여부(survived)가 있으며, target class 는 0, 1로 이루어져 있으므로 둘 중에 하나를 골라내는 이진 분류(binary classfication) 문제이다.
우리가 지금 가지고 있는 train set 의 survived를 제외한 input 을 가지고 모델을 최적화시켜서 각 샘플(탑승객)의 생존유무를 판단하는 모델을 만들어 내는 것이 목표이다.
그 후 모델이 학습하지 않았던 test set 을 input 으로 주어서 test set 의 각 샘플(탑승객)의 생존 유무를 예측한다.

### 4.1 Preparation - Split dataset into train, valid, test set

가장 먼저, 학습에 쓰일 데이터와 target label 즉 생존유무를 분리해 내는 과정으로써 drop을 사용하여 코드를 구성한다.

In [36]:
X_train = df_train.drop('Survived', axis=1).values
target_label = df_train['Survived'].values
X_test = df_test.values

* 보통 train, test만 언급되지만, 실제 좋은 모델을 만들기 위해서 우리는 valid set을 따로 만들어 모델 평가를 해 볼 것이다.

* 마치 축구대포팀이 팀훈련(train)을 하고 바로 월드컵(test)으로 나가는 것이 아니라, 팀훈련(train)을 한 다음 평가전(valid)을 거쳐 팀의 훈련 정도를 확인하고 월드컵(test)에 나가는 것과 비슷한 이치이다.  

* 훈련 세트와 테스트 세트를 분리하는 이유는 연습 문제와 시험 문제가 달라야 올바르게 학생을 평가할 수 있듯이 모델을 평가할 때도 훈련 때 사용한 데이터가 아닌 훈련 때 사용하지 않은 데이터를  평가를 해야 올바르게 평가가 되기 때문이다. 즉 모델을 정확히 검증하기 위해 거치는 과정인 셈이다. 

* 훈련 세트와 테스트 세트를 분리하는 것은 train_test_split을 사용하면 된다. 앞으로 모델링을 하면서 훈련 세트와 테스트 세트를 분리하는 과정을 필히 거치게 되므로, 이 구문은 지겹도록 보게 될 것으로 보인다.

In [37]:
X_tr, X_vld, y_tr, y_vld = train_test_split(X_train, target_label, test_size=0.3, random_state=2018)

* sklearn 에서는 여러 머신러닝 알고리즘을 지원해 준다. 열거하기엔 너무 많으므로, 직접 documentation에 들어가 보시길 추천합니다.  

* http://scikit-learn.org/stable/supervised_learning.html#supervised-learning 여기에 들어가서 확인해보자. 사이킷런 공식 사이트인데 굉장히 많은 알고리즘이 지원됨을 알 수 있다. 우리는 머신러닝을 배울 때 이 사이트에서 많은 부분을 활용할 것으로 기대된다.

* 본 튜토리얼에서는 랜덤포레스트 모델을 사용하고자 한다. 랜덤포레스트는 결정 트리 기반 모델이며, 여러 결정 트리들을 앙상블한 모델이라고 한다.

* 각 머신러닝 알고리즘에는 여러 파라미터들이 있다. 랜덤포레스트분류기도 n_estimators, max_features, max_depth, min_samples_split, min_samples_leaf 등 여러 파라미터들이 존재한다. 이것들이 어떻게 세팅되냐에 따라 같은 데이터셋이라 하더라도 모델의 성능이 달라지게 된다.  

* 파라미터 튜닝은 시간, 경험, 알고리즘에 대한 이해 등이 필요하고, 결국 많이 써봐야 모델도 잘 세울 수 있다. 캐글 필사를 하는 이유가 바로 많이 써봄으로써 모델을 세우는 데 익숙해질 수 있기 때문이고 작성을 하고 있는 지금도 조금씩 체감을 하고 있다. 여러 데이터셋을 가지고 모델을 이리저리 써봐야 튜닝하는 감이 생길테니까!

* 일단 지금은 튜토리얼이니 파라미터 튜닝은 잠시 제쳐두기로 하고, 기본 default 세팅으로 진행코자 한다. 모델 객체를 만들고, fit 메서드를 활용해 모델을 학습시킨다. fit 역시 모델을 학습할 때 반드시 포함이 되는 구문이니 잘 익혀 둬야 겠다고 생각했다.

* 그런 후 valid set input 을 넣어주어 예측값(X_vld sample(탑승객)의 생존여부)를 얻어내면 된다.

### 4.2 Model generation and prediction

In [45]:
model = RandomForestClassifier()
model.fit(X_tr, y_tr)
prediction = model.predict(X_vld)

* 단 세 줄만으로 모델을 세우고 예측까지 해보는 코드를 작성했다. 단, 작성할 때 변수 이름에 유의해야 된다는 점!! fit 뒤에는 훈련 tr 혹은 train이 들어간 변수가 주로 들어가고, predict 뒤로는 test 혹은 valid, vld와 같은 단어가 포함된 경우가 많다는 점이 있다. 이 점은 코드를 작성할 때 참고할 만한 내용이라고 생각했다.

* 성능 모델을 확인하기 위해 다음 코드를 작성해서 결과를 확인해 보자.

In [44]:
print('총 {}명 중 {:.2f}% 정확도로 생존을 맞춤'.format(y_vld.shape[0], 100 * metrics.accuracy_score(prediction, y_vld)))

아무런 파라미터 튜닝도 없이 생으로 모델 검증을 했음에도 82%의 정확도로 생존을 맞혔음을 알 수 있다. 결과값은 코드를 돌릴 때마다 80% 대에서 약간의 차이는 있을 수 있다는 점도 밝혀냈다.

### 4.3 Feature importance

* 학습된 모델은 feature importance를 가지게 되는데, 우리는 이것을 확인하여 지금 만든 모델이 어떤 feature에 영향을 많이 받았는 지 확인할 수 있다.

* 쉽게 말해, $10 = 4x_1 + 2x_2 + 1*x_3$ 을 생각하면, 우리는 $x_1$이 결과값(10)에 큰 영향을 준다고 생각 할 수 있다. feature importance 는 4, 2, 1 을 이야기하며, $x_1$이 가장 큰 값(4)를 가지므로, 이 모델에 가장 큰 영향을 미친다고 말할 수 있게 되는 것이다.

* 학습된 모델은 기본적으로 feature importances를 가지고 있어서 쉽게 그 수치를 얻을 수 있기는 하다. pandas series 를 이용하면 쉽게 sorting(정렬)을 하여 그래프를 그릴 수 있습니다.

In [46]:
from pandas import Series

feature_importance = model.feature_importances_
Series_feat_imp = Series(feature_importance, index=df_test.columns)
plt.figure(figsize=(8, 8))

Series_feat_imp.sort_values(ascending=True).plot.barh()
plt.xlabel('Feature importance')
plt.ylabel('Feature')
plt.show()

* 우리가 얻은 모델에서는 Fare(요금)가 가장 큰 영향력을 가지며, 그 뒤로 Initial_2, Age_cat, Pclass가 차례로 중요도를 가진다.

* 사실 feature importance는 지금 모델에서의 importance를 나타낸다. 만약 다른 모델을 사용하게 된다면 feature importance가 다르게 나올 수 있다.

* 이 feature importance를 보고 실제로 Fare가 중요한 feature일 수 있다고 판단을 내릴 수는 있지만, 이것은 결국 모델에 귀속되는 하나의 결론이므로 통계적으로 좀 더 살펴보긴 해야할 것이다. 결국 절대적인 개념이 아닌 상대적인 개념이라는 것이다.

* featuure importanc 를 가지고 좀 더 정확도가 높은 모델을 얻기 위해 feature selection을 할 수도 있고, 좀 더 빠른 모델을 위해 feature 제거를 할 수 있다.

### 4.4 Prediction on Test set

In [ ]:
이제 모델이 학습하지 않았던(보지 않았던) 테스트셋을 모델에 주어서, 생존여부를 예측해보고자 한다. 이 결과는 실제로 submission(제출용) 이므로 결과는
leaderboard에서 확인할 수 있다. 캐글에서 준 파일, gender_submission.csv 파일을 읽어서 제출 준비를 하고자 한다.

In [51]:
submission = pd.read_csv('../input/titanic/gender_submission.csv')
submission.head()

이제 testset에 대하여 예측을 하고, 결과를 csv 파일로 저장하는 과정이다.

In [52]:
prediction = model.predict(X_test)
submission['Survived'] = prediction
submission.to_csv('./my_first_submission.csv', index=False)

이름과 일치하는 해당 파일을 찾은 후 캐글에 제출하면 끝!!

# 5. 회고

titanic 예제를 가지고 굉장히 멀리까지 왔는데 앞으로 배울 것이 너무나도 많고 활용 범위가 무궁무진하다고 하니 기대 반 걱정 반이다.

좀 더 참신한 feature engineering, 머신 러닝 모델, hyperparameter tunning, ensembling 등 너무나 많다고 한다. 아직 이게 무슨 개념인지도 모르겠지만 머신 러닝, 딥러닝을 공부하면서 차차 알아갈 것이라고 한다. 포기하지 말고 재미있게 공부하며 성장하는 나 자신을 바라보며 회고를 마치고자 한다!!!